In [1]:

from get_books import get_many_books

# Import some books and pull them all into one giant string
book_ids = [15, 63, 16, 41, 14, 76, 99, 209, 110, 9]
dataset = get_many_books(book_ids, data_temp="./data/gutenberg_data")
rawtext = ""
for book in dataset:
    rawtext += book

Getting book 15...
	1218778 characters read
Getting book 63...
	103137 characters read
Getting book 16...
	255644 characters read
Getting book 41...
	69837 characters read
Getting book 14...
	1897512 characters read
Getting book 76...
	571137 characters read
Getting book 99...
	45748 characters read
Getting book 209...
	228531 characters read
Getting book 110...
	841270 characters read
Getting book 9...
	21311 characters read


In [2]:
import huffman
import json

# Create a huffman codebook
codebook = huffman.codebook((char, rawtext.count(char)) for char in set(rawtext))

# Save codebook to file
with open("huffman_codebook.json", "w") as f:
    json.dump(codebook, f)

from huffman_encoder import huffman_scramble


In [3]:
from llist import dllist
from collections import Counter
import heapq

def initialize_pair_counter(linked_list):
    pair_counter = Counter()
    node = linked_list.first
    while node and node.next:
        pair = node.value + node.next.value
        pair_counter[pair] += 1
        node = node.next
    return pair_counter

def build_heap(pair_counter):
    # Python heapq is min-heap, so use negative counts for max-heap behavior
    return [(-count, pair) for pair, count in pair_counter.items()]

def BPE(tokens, num_merges):
    linked_list = dllist(tokens)
    pair_counter = initialize_pair_counter(linked_list)
    heap = build_heap(pair_counter)
    heapq.heapify(heap)

    i = 0
    while i < num_merges:
        
        count, to_replace = heapq.heappop(heap)
        if pair_counter[to_replace] != -count:
            if pair_counter[to_replace] > 0:
                heapq.heappush(heap, (-pair_counter[to_replace], to_replace))
            continue

        print(f"Iteration {i + 1}: merging '{to_replace}' ({-count} occurrences)")

        node = linked_list.first
        while node and node.next:
            if node.value + node.next.value == to_replace:
                second_part = node.next

                # Update counters before replacing
                if node.prev:
                    left_pair = node.prev.value + node.value
                    pair_counter[left_pair] -= 1
                if second_part.next:
                    right_pair = second_part.value + second_part.next.value
                    pair_counter[right_pair] -= 1

                # Insert merged token
                pair_counter[to_replace] -= 1
                merged_node = linked_list.insertafter(to_replace, second_part)
                linked_list.remove(second_part)
                linked_list.remove(node)

                # Update counters after replacing
                if merged_node.prev:
                    new_left = merged_node.prev.value + merged_node.value
                    pair_counter[new_left] += 1
                    heapq.heappush(heap, (-pair_counter[new_left], new_left))
                if merged_node.next:
                    new_right = merged_node.value + merged_node.next.value
                    pair_counter[new_right] += 1
                    heapq.heappush(heap, (-pair_counter[new_right], new_right))

                node = merged_node.next
            else:
                node = node.next
            
        i += 1

    return linked_list

# Turn into list of chars for BPE
words = list(huffman_scramble(rawtext, codebook))

# Get the tokens that have been made through BPE merging
tokens = BPE(words, 5000)

# Step 1: Count the frequency of each unique token
frequency = Counter(tokens)

# Step 2: Sort the tokens by frequency (in descending order)
sorted_items = sorted(frequency.items(), key=lambda x: x[1], reverse=True)

# Step 3: Create a dictionary where each token is mapped to an index based on its frequency rank
token_to_index = {item: index for index, (item, _) in enumerate(sorted_items)}

with open("compressed_tokenizer_dict.json", "w") as f:
    json.dump(token_to_index, f)

Iteration 1: merging 'Ón' (5767 occurrences)
Iteration 2: merging '¦Ý' (4720 occurrences)
Iteration 3: merging 'M»' (4629 occurrences)
Iteration 4: merging 'v' (4230 occurrences)
Iteration 5: merging 's' (3497 occurrences)
Iteration 6: merging 'Îp' (3299 occurrences)
Iteration 7: merging 'é·' (3286 occurrences)
Iteration 8: merging '9Ã' (3262 occurrences)
Iteration 9: merging 'á' (3250 occurrences)
Iteration 10: merging 'i·' (2771 occurrences)
Iteration 11: merging 'ÊÖ' (2560 occurrences)
Iteration 12: merging '6í' (2225 occurrences)
Iteration 13: merging 'ç8' (2141 occurrences)
Iteration 14: merging '#o' (2111 occurrences)
Iteration 15: merging 'FÞ' (2063 occurrences)
Iteration 16: merging '6ì' (2044 occurrences)
Iteration 17: merging '' (1943 occurrences)
Iteration 18: merging '0' (1925 occurrences)
' (1913 occurrences)g 'ç
Iteration 20: merging '­' (1863 occurrences)
Iteration 21: merging 'Ûm' (1862 occurrences)
Iteration 22: merging 'Á' (1790 occurrences)
Iteration 23: merg

In [4]:
with open("compressed_tokenizer_dict.json", "w") as f:
    json.dump(token_to_index, f)